In [12]:
import boto3
import time
import sagemaker
import os 
import trp
import pandas as pd
from util.preprocess import *

In [16]:
fileName =  'Untitled document.pdf'
textractObjectName = os.path.join('files', fileName)

In [17]:
# Retrieve the list of existing buckets
s3 = boto3.client('s3')
response = s3.list_buckets()

# Output the bucket names
bucket = response['Buckets'][3]

In [18]:
textract = boto3.client('textract')
response = textract.start_document_analysis(
    DocumentLocation={
        'S3Object': {
            'Bucket': bucket["Name"],
            'Name': textractObjectName
        }},
    FeatureTypes=[
        'TABLES',
    ]
    )

textractJobId = response["JobId"]
print('job id is: ',textractJobId)

job id is:  b7411b6c4bcee7371fc5416ff40d5a8a55568bc3a151269c7e298535fb94d7fb


In [19]:
%%time
time.sleep(5)
response = textract.get_document_analysis(JobId=textractJobId)
status = response["JobStatus"]

while(status == "IN_PROGRESS"):
    time.sleep(5)
    response = textract.get_document_analysis(JobId=textractJobId)
    status = response["JobStatus"]
    print("Textract Job status: {}".format(status))

Textract Job status: SUCCEEDED
CPU times: user 45.1 ms, sys: 8.79 ms, total: 53.9 ms
Wall time: 11.2 s


In [20]:
%%time
pages = []

time.sleep(5)

response = textract.get_document_analysis(JobId=textractJobId)

pages.append(response)

nextToken = None
if('NextToken' in response):
    nextToken = response['NextToken']

while(nextToken):
    time.sleep(5)

    response = textract.get_document_analysis(JobId=textractJobId, NextToken=nextToken)

    pages.append(response)
    print("Resultset page recieved: {}".format(len(pages)))
    nextToken = None
    if('NextToken' in response):
        nextToken = response['NextToken']

CPU times: user 47.1 ms, sys: 10.6 ms, total: 57.7 ms
Wall time: 6.32 s


In [21]:
doc = trp.Document(pages)
print("Total length of document is",len(doc.pages))
idx=1
for page in doc.pages:
    print(color.BOLD + f"Results from page {idx}: \n" + color.END, page.text)
    idx=idx+1

Total length of document is 2
Results from page 1: 
 PREOPERATIVE DIAGNOSIS: Medial meniscal tear of the right knee. POSTOPERATIVE
DIAGNOSES: 1. Medial meniscal tear, right knee.,2. Lateral meniscal tear, right knee., 3.
Osteochondral lesion, medial femoral condyle, right knee. 4. Degenerative joint disease, right
knee., 5. Patella grade-II chondromalacia., 6. Lateral femoral condyle grade II-III
chondromalacia. PROCEDURE PERFORMED: 1. Arthroscopy, right knee., 2. Medial
meniscoplasty, right knee., 3. Lateral meniscoplasty, right knee. 4. Medial femoral
chondroplasty, right knee., 5. Medical femoral microfracture, right knee., 6. Patellar
chondroplasty.,7. Lateral femoral chondroplasty.,ANESTHESIA: General. ,,ESTIMATED
BLOOD LOSS: , Minimal.,COMPLICATIONS: None.,BRIEF HISTORY AND INDICATION FOR
PROCEDURE: The patient is a 47-year-old female who has knee pain since 03/10/03 after
falling on ice. The patient states she has had inability to bear significant weight and had
swelling, poppin

In [22]:
maxLength=20000

comprehendResponse = []
comprehend_medical_client = boto3.client(service_name='comprehendmedical', region_name='us-east-1')

for page in doc.pages:
    pageText = page.text
    
    for i in range(0, len(pageText), maxLength):
        response = comprehend_medical_client.detect_entities_v2(Text=pageText[0+i:maxLength+i])
        comprehendResponse.append(response)
    patient_string = ""